<a href="https://colab.research.google.com/github/jurandi82/Python/blob/main/Otimizacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 <font size="6">Testes de otimização</font>

Usando Numba.Jit e Cython **Magic**

#Preparando o ambiente

In [1]:
#para poder utilizar o jit
from numba import jit

#para medir o tempo de execução
from time import time

#para produção dos valores de exemplo
import numpy as np

#extras para visualização
from tqdm.notebook import trange

In [2]:
%load_ext Cython
#Cython é uma extenção, precisa ser carregada dessa forma.

# Valores para teste

In [3]:
valores = np.random.rand(1_000)
loops = 1_000

#Criando funções

In [4]:
#Python Puro
def media_mv(x, w=3):
  r =[]
  for i in range(len(x)):
    if i+1 < w:
      r.append(0)
    else:
      r.append((x[i-w+1:i+1]).sum()/w)
  return r

In [5]:
%%cython
#Cython cell
def media_mv2(x, w=3):
  r =[]
  for i in range(len(x)):
    if i+1 < w:
      r.append(0)
    else:
      r.append((x[i-w+1:i+1]).sum()/w)
  return r


In [6]:
#Usando Numba.jit

@jit
def media_mv3(x, w=3):
  r =[]
  for i in range(len(x)):
    if i+1 < w:
      r.append(0)
    else:
      r.append((x[i-w+1:i+1]).sum()/w)
  return r

#Testanto numero de interações

In [14]:
print('Teste 1 - Python Puro')
i=1
inicio = time()
while(time()-inicio<2):
  _ = media_mv(valores)
  i+=1
print(f'Interações = {i}')

Teste 1 - Python Puro
Interações = 545


In [15]:
print('Teste 2 - com Cython')
i=1
inicio = time()
while(time()-inicio<2):
  _ = media_mv2(valores)
  i+=1
print(f'Interações = {i}')

Teste 2 - com Cython
Interações = 596


In [16]:
print('Teste 3 - com JIT')
i=1
inicio = time()
while(time()-inicio<2):
  _ = media_mv3(valores)
  i+=1
print(f'Interações = {i}')

Teste 3 - com JIT
Interações = 25614


#Testanto tempo de execução em loop

In [10]:
%%time
print('Teste 1 - Python Puro')
inicio = time()
for i in trange(loops):
  _ = media_mv(valores)
tempo1=time()-inicio

Teste 1 - Python Puro


  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 4.03 s, sys: 32.6 ms, total: 4.06 s
Wall time: 4.15 s


In [11]:
%%time
print('Teste 2 - com Cython')
inicio = time()
for i in trange(loops):
  _ = media_mv2(valores)
tempo2=time()-inicio

Teste 2 - com Cython


  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 3.81 s, sys: 35 ms, total: 3.85 s
Wall time: 3.9 s


In [12]:
%%time
print('Teste 3 - com JIT')
inicio = time()
for i in trange(loops):
  _ = media_mv3(valores)
tempo3=time()-inicio

Teste 3 - com JIT


  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 136 ms, sys: 12.5 ms, total: 148 ms
Wall time: 161 ms


In [13]:
print(f'Economia de tempo  Cython contra Python Puro {round((1-tempo2/tempo1)*100,2)}%')
print(f'Economia de tempo  JIT    contra Cython      {round((1-tempo3/tempo2)*100,2)}%')
print(f'Economia de tempo  JIT    contra Python Puro {round((1-tempo3/tempo1)*100,2)}%')

Economia de tempo  Cython contra Python Puro 5.86%
Economia de tempo  JIT    contra Cython      95.88%
Economia de tempo  JIT    contra Python Puro 96.12%
